# MCP - Conoce el MCP inspector!

### Notebook Credits: 
> DeepLearning.ai [Full Free Course](https://learn.deeplearning.ai/courses/mcp-build-rich-context-ai-apps-with-anthropic)

> YouTube [Promo Video](https://www.youtube.com/watch?v=IkBslShH5to)

In [1]:
from pathlib import Path
Path('mcp_assets_01').mkdir(exist_ok=True)

In [2]:
%%writefile mcp_assets_01/research_server.py

import arxiv
import json
import os
from typing import List
from mcp.server.fastmcp import FastMCP

PAPER_DIR = "papers"

# Initialize FastMCP server
mcp = FastMCP("research")

@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

@mcp.tool()
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Writing mcp_assets_01/research_server.py


#### Prerequisites!
- Install [uv](https://docs.astral.sh/uv/getting-started/installation/)
- You need to make sure you have `Node.js` installed on your machine. This is needed to run packages written in  `Typescript` (like the MCP inspector and maybe other MCP servers that you may download to use in your app). For instructions, check [here](https://docs.npmjs.com/downloading-and-installing-node-js-and-npm)

### Instructions
- To open the terminal, run the cell below.
- Deactivate the existing Virtual Env
    - `deactivate`
- Navigate to the project directory and initiate it with `uv`:
    ```
    deactivate
    cd build_bedrock_mcp/mcp_assets_01
    python3 -m venv .venv                  # Create new virtual env
    source .venv/bin/activate              # Activate it
    echo $VIRTUAL_ENV                      # Make sure it's loaded
    python3 -m pip install uv              # Install UV in the virtual env
    uv init                                # Now use UV commands
    uv add mcp arxiv
    npx @modelcontextprotocol/inspector uv run research_server.py
    ```
    - If you get a message asking "need to install the following packages", type: `y`
- You will get a message saying that the inspector is up and running at a specific address. To open the inspector, click on that given address. The inspector will open in another tab.
- If you tested the tool and would like to access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L4` -> `mcp_assets_01`.
- Once you're done with the inspector UI, make sure to close the inspector by typing `Ctrl+C` in the terminal below.

In the inspector UI, make sure you have:
1.  `uv` under command
2.  `run research_server.py` under arguments